In [1]:
import geopandas as gpd
from h3 import h3
import pandas as pd
from shapely.geometry import Point, MultiPoint, Polygon
from tqdm import tqdm

In [2]:
# Explode MultiPoint geometries into individual Point rows
def explode_multipoints(gdf):
    rows = []
    for idx, row in tqdm(gdf.iterrows()):
        geom = row.geometry
        if isinstance(geom, MultiPoint):
            for pt in geom.geoms:
                new_row = row.copy()
                new_row.geometry = pt
                rows.append(new_row)
        elif isinstance(geom, Point):
            rows.append(row)
        else:
            continue  # skip other geometry types
    return gpd.GeoDataFrame(rows, columns=gdf.columns, crs=gdf.crs)

Let's aggregate them just spatially:

In [3]:
for region in [#"california", 
               "central_europe"]:

    # Load your GeoPackage file
    gdf = gpd.read_parquet(f"Data/bsky_{region}_gdf_esda.parquet")

    # Ensure it's using WGS84 (latitude/longitude) coordinates
    gdf = gdf.to_crs(epsg=4326)

    # explode multipoints
    gdf_points = explode_multipoints(gdf)

    gdf_points_relevant = gdf_points[gdf_points['disaster_related'] == 1]
    
    for h3_level in tqdm([4, 5, 6]): 
        h3_grid = gpd.read_file(f"Data/h3 grids/h3_{region}_level_{h3_level}.gpkg")
        h3_grid = h3_grid.to_crs(4326)

        # Perform spatial join (use 'inner' join to avoid NaNs initially)
        joined = gpd.sjoin(gdf_points, h3_grid, how='inner', predicate='intersects')
        joined_relevant = gpd.sjoin(gdf_points_relevant, h3_grid, how='inner', predicate='intersects')

        # Count number of points per H3 cell
        counts = joined.groupby("H3HASH").size().reset_index(name='count')
        counts_relevant = joined_relevant.groupby("H3HASH").size().reset_index(name='count')

        # Join with original H3 geometries (outer join to keep all grid cells)
        post_aggregated = h3_grid.merge(counts, on='H3HASH', how='left')
        post_aggregated_relevant = h3_grid.merge(counts_relevant, on='H3HASH', how='left')

        # Fill empty grid cells with zero
        post_aggregated['count'] = post_aggregated['count'].fillna(0).astype(int)
        post_aggregated_relevant['count'] = post_aggregated_relevant['count'].fillna(0).astype(int)

        # Save to GeoPackage
        post_aggregated.to_crs(epsg=3857).to_file(
            f"Results/bluesky_posts_{region}_h3_level_{h3_level}.gpkg", 
            driver="GPKG"
        )

        post_aggregated_relevant.to_crs(epsg=3857).to_file(
            f"Results/bluesky_relevant_posts_{region}_h3_level_{h3_level}.gpkg", 
            driver="GPKG"
        )


105767it [00:13, 7731.22it/s]
100%|██████████| 3/3 [02:42<00:00, 54.22s/it]


Let's also consider time by dividing the dataframe on a weekly basis:

In [ ]:
for region in ["california", 
            #    "central_europe"
               ]:

    # Load your GeoPackage file
    gdf = gpd.read_parquet(f"Data/bsky_{region}_gdf_esda.parquet")

    # Ensure it's using WGS84 (latitude/longitude) coordinates
    gdf = gdf.to_crs(epsg=4326)

    # explode multipoints
    gdf_points = explode_multipoints(gdf)

    # Add a week number column based on a datetime column
    gdf_points['createdAt'] = pd.to_datetime(gdf_points['createdAt'], utc=True)
    gdf_points['week'] = pd.to_datetime(gdf_points['createdAt']).dt.isocalendar().week

    gdf_points_relevant = gdf_points[gdf_points['disaster_related'] == 1]
    
    for h3_level in tqdm([4, 5]): 
        h3_grid = gpd.read_file(f"Data/h3 grids/h3_{region}_level_{h3_level}.gpkg")
        h3_grid = h3_grid.to_crs(4326)

        for week_number in sorted(gdf_points['week'].unique()):
            week_points = gdf_points[gdf_points['week'] == week_number]
            week_relevant_points = gdf_points_relevant[gdf_points_relevant['week'] == week_number]

            # Spatial join
            joined = gpd.sjoin(week_points, h3_grid, how='inner', predicate='intersects')
            joined_relevant = gpd.sjoin(week_relevant_points, h3_grid, how='inner', predicate='intersects')

            # Count points per H3 cell
            counts = joined.groupby("H3HASH").size().reset_index(name='count')
            counts_relevant = joined_relevant.groupby("H3HASH").size().reset_index(name='count')

            # Merge with grid
            post_aggregated = h3_grid.merge(counts, on='H3HASH', how='left')
            post_aggregated_relevant = h3_grid.merge(counts_relevant, on='H3HASH', how='left')

            # Fill NaNs
            post_aggregated['count'] = post_aggregated['count'].fillna(0).astype(int)
            post_aggregated_relevant['count'] = post_aggregated_relevant['count'].fillna(0).astype(int)

            # Save with week suffix
            post_aggregated.to_crs(epsg=3857).to_file(
                f"Results/weekly/bluesky_posts_{region}_h3_level_{h3_level}_week_{week_number}.gpkg", 
                driver="GPKG"
            )

            post_aggregated_relevant.to_crs(epsg=3857).to_file(
                f"Results/weekly/bluesky_relevant_posts_{region}_h3_level_{h3_level}_week_{week_number}.gpkg", 
                driver="GPKG"
            )

570570it [01:45, 5419.41it/s]


,cid,uri,author_displayName,author_handle,author_did,createdAt,langs,text,replyCount,repostCount,...,p_fear,p_joy,p_sadness,anger,fear,joy,sadness,no_emotion,disaster_related,week
15,bafyreiai5hex6t2wr2npiuy5ql5lnsfjacemieu6cjjta...,at://did:plc:2amx7tvyqv2nxfqvp5ycskmr/app.bsky...,Muneeb,viralltoday.com,did:plc:2amx7tvyqv2nxfqvp5ycskmr,2024-12-24 00:26:53+00:00,None,"Santa Cruz wharf section collapses, sending 3 ...",0,0,...,0.590658,0.046475,0.321727,False,True,False,False,False,1,52
15,bafyreiai5hex6t2wr2npiuy5ql5lnsfjacemieu6cjjta...,at://did:plc:2amx7tvyqv2nxfqvp5ycskmr/app.bsky...,Muneeb,viralltoday.com,did:plc:2amx7tvyqv2nxfqvp5ycskmr,2024-12-24 00:26:53+00:00,None,"Santa Cruz wharf section collapses, sending 3 ...",0,0,...,0.590658,0.046475,0.321727,False,True,False,False,False,1,52
15,bafyreiai5hex6t2wr2npiuy5ql5lnsfjacemieu6cjjta...,at://did:plc:2amx7tvyqv2nxfqvp5ycskmr/app.bsky...,Muneeb,viralltoday.com,did:plc:2amx7tvyqv2nxfqvp5ycskmr,2024-12-24 00:26:53+00:00,None,"Santa Cruz wharf section collapses, sending 3 ...",0,0,...,0.590658,0.046475,0.321727,False,True,False,False,False,1,52
29,bafyreigljnz5z2d5te2t4nvoh5gd7cjspxgijxlzuhxwv...,at://did:plc:jwmyh3wycmjqf4cav3e5iazu/app.bsky...,DJ Black,djblack21.bsky.social,did:plc:jwmyh3wycmjqf4cav3e5iazu,2024-12-24 00:25:53.210000+00:00,[en],It happened around 12:45 p.m. Monday at Santa ...,0,1,...,0.646496,0.037827,0.190053,False,True,False,False,False,1,52
37,bafyreih7lxoxueby4hdcbiuu33kjimoxr5dlny7ebgvt6...,at://did:plc:ykbvqikdxucwe4hsmbzltrbk/app.bsky...,,butterfliesfree.bsky.social,did:plc:ykbvqikdxucwe4hsmbzltrbk,2024-12-24 00:24:42.005000+00:00,[da],Victoria prepares for worst fire conditions si...,0,0,...,0.387715,0.030460,0.059179,True,False,False,False,False,1,52


100%|██████████| 2/2 [02:58<00:00, 89.25s/it]
